## Projekt zaawansowane metody uczenia maszynowego
### Anna Kozioł 

Celem projektu było próba przewidzenia czy dany użytkownik kliknie w rekamę.

In [1]:
from urllib.request import urlretrieve
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from pprint import pprint
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub

In [2]:
def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

d100k_url = "https://home.ipipan.waw.pl/sj/ml/AdClick/D100k.tsv.gz"
D5M_test_x_url = "https://home.ipipan.waw.pl/sj/ml/AdClick/D5M_test_x.tsv.gz"
D10M_url = "https://home.ipipan.waw.pl/sj/ml/AdClick/D10M.tsv.gz"
 
#download(d100k_url,'D100k.tsv.gz')
download(D5M_test_x_url,'D5M_test_x.tsv.gz')

print("All the files are downloaded")

Download file... D5M_test_x.tsv.gz ...
File downloaded
All the files are downloaded


In [43]:
download(D10M_url,'D10M.tsv.gz')

Download file... D10M.tsv.gz ...
File downloaded


In [3]:
download(d100k_url,'D100k.tsv.gz')

Download file... D100k.tsv.gz ...
File downloaded


In [4]:
ds = tf.data.experimental.make_csv_dataset("D100k.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                            batch_size = 100, # increase for real work
                                           num_epochs=1,
                                           label_name="Click")

In [44]:
ds5M = tf.data.experimental.make_csv_dataset("D10M.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                            batch_size = 100, # increase for real work
                                           num_epochs=1,
                                           label_name="Click")

In [5]:
ds_testowy=tf.data.experimental.make_csv_dataset("D5M_test_x.tsv.gz",
                                              column_defaults=[tf.int32, tf.float32, tf.int32, 
                                                               tf.int32, tf.int32,tf.int32,tf.int32,
                                                               tf.int32,tf.int32, tf.string,tf.string,
                                                               tf.string, tf.string],
                                           field_delim="\t",
                                           compression_type="GZIP",
                                           batch_size=1000,  # increase for real work
                                           label_name="Click",
                                           num_epochs=1,
                                           ignore_errors=True)

In [6]:
pprint(ds.element_spec)

(OrderedDict([('DisplayURL',
               TensorSpec(shape=(None,), dtype=tf.float32, name=None)),
              ('AdId', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('AdvertiserId',
               TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Depth', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Position',
               TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('UserID', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Gender', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('Age', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
              ('AdKeyword_tokens',
               TensorSpec(shape=(None,), dtype=tf.string, name=None)),
              ('AdTitle_tokens',
               TensorSpec(shape=(None,), dtype=tf.string, name=None)),
              ('AdDescription_tokens',
               TensorSpec(shape=(None,), dtype=tf.string, 

## zbiór treningowy i testowy 

In [6]:
train = ds.take(700) 

In [7]:
test = ds.skip(700)

In [46]:
train5M = ds5M.take(700) 

In [47]:
test5M = ds5M.skip(700)

## Transformacje kolumn

In [45]:
columns = []

age_discr = tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list('Age', [1,2,3,4,5,6])
    )

columns.append(age_discr)

gender_discr = tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list('Gender', [0,1,2])
    )

columns.append(gender_discr)

position_discr = tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list('Position', [1,2,3])
    )
columns.append(position_discr)


depth_discr = tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list('Depth', [1,2,3])
    )
columns.append(depth_discr)

thal_hashed =  tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_hash_bucket(
      'AdDescription_tokens', hash_bucket_size=1000)
)

columns.append(thal_hashed)


hashed2 =  tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_hash_bucket(
      'Query_tokens', hash_bucket_size=500)
)

columns.append(hashed2)

hashed3 =  tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_hash_bucket(
      'AdTitle_tokens', hash_bucket_size=500)
)

columns.append(hashed3)








In [48]:
columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Age', vocabulary_list=(1, 2, 3, 4, 5, 6), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Gender', vocabulary_list=(0, 1, 2), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Position', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Depth', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=HashedCategoricalColumn(key='AdDescription_tokens', hash_bucket_size=1000, dtype=tf.string)),
 IndicatorColumn(categorical_column=HashedCategoricalColumn(key='Query_tokens', hash_bucket_size=500, dtype=tf.string)),
 IndicatorColumn(categorical_column=HashedCategoricalColumn(key='AdTitle_tokens', hash

## Model


In [10]:
feature_layer  = tf.keras.layers.DenseFeatures(columns)
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(100, activation='relu'),
  layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])
''
model.fit(train,       
          epochs=5, validation_data = test)

Epoch 1/5
700/700 [==============================] - 9s 13ms/step - loss: 0.2024 - accuracy: 0.9504 - auc: 0.5881 - val_loss: 0.1985 - val_accuracy: 0.9503 - val_auc: 0.6275
Epoch 2/5
700/700 [==============================] - 9s 13ms/step - loss: 0.1776 - accuracy: 0.9508 - auc: 0.7376 - val_loss: 0.1908 - val_accuracy: 0.9519 - val_auc: 0.6531
Epoch 3/5
700/700 [==============================] - 9s 13ms/step - loss: 0.1564 - accuracy: 0.9514 - auc: 0.8336 - val_loss: 0.1941 - val_accuracy: 0.9526 - val_auc: 0.6731
Epoch 4/5
700/700 [==============================] - 9s 13ms/step - loss: 0.1221 - accuracy: 0.9592 - auc: 0.9110 - val_loss: 0.1953 - val_accuracy: 0.9537 - val_auc: 0.6919
Epoch 5/5
700/700 [==============================] - 9s 13ms/step - loss: 0.0889 - accuracy: 0.9696 - auc: 0.9540 - val_loss: 0.2073 - val_accuracy: 0.9456 - val_auc: 0.7041


In [49]:
feature_layer  = tf.keras.layers.DenseFeatures(columns)
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(100, activation='relu'),
  layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])

model.fit(train5M,       
          epochs=5, validation_data = test5M)

Epoch 1/5
700/700 [==============================] - 487s 696ms/step - loss: 0.2033 - accuracy: 0.9503 - auc: 0.5874 - val_loss: 0.1887 - val_accuracy: 0.9532 - val_auc: 0.5959
Epoch 2/5
700/700 [==============================] - 484s 692ms/step - loss: 0.1778 - accuracy: 0.9509 - auc: 0.7375 - val_loss: 0.1915 - val_accuracy: 0.9532 - val_auc: 0.5901
Epoch 3/5
700/700 [==============================] - 486s 694ms/step - loss: 0.1586 - accuracy: 0.9514 - auc: 0.8229 - val_loss: 0.2003 - val_accuracy: 0.9532 - val_auc: 0.5708
Epoch 4/5
700/700 [==============================] - 502s 717ms/step - loss: 0.1300 - accuracy: 0.9562 - auc: 0.8963 - val_loss: 0.2178 - val_accuracy: 0.9497 - val_auc: 0.5661
Epoch 5/5
700/700 [==============================] - 483s 690ms/step - loss: 0.0956 - accuracy: 0.9678 - auc: 0.9483 - val_loss: 0.2459 - val_accuracy: 0.9436 - val_auc: 0.5721


In [12]:
pwstwa = model.predict(ds_testowy )

In [14]:
pwstwa

array([[0.06295416],
       [0.2642166 ],
       [0.08379552],
       ...,
       [0.07893473],
       [0.10833943],
       [0.49144122]], dtype=float32)

In [36]:
wyniki = pd.DataFrame(pwstwa)

In [37]:
wyniki.columns = ["AK"]

In [38]:
print(wyniki.to_csv(sep='\t', index=False))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [41]:
wyniki.to_csv('scory_projekt.txt', sep='\t',  index=None)